<a href="https://colab.research.google.com/github/lisaong/stackup-workshops/blob/text_similarity_datasource/text-similarity/text_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import spacy
from sklearn.decomposition import PCA

In [0]:
# load the english language model
nlp = spacy.load("en_core_web_sm")

In [9]:
# try it out with some text
tokens = nlp("dog cat banana lah")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov, token.vector)

dog True 19.375494 True [ 1.0273597  -0.8354976  -0.9416786  -0.8869799   1.4221182  -1.4879098
 -1.4275644   2.3585858   1.4182274   4.125562    4.643529    3.025484
  4.6832633  -0.8770789   0.93978465 -1.102579   -0.9267146   1.4311049
 -1.476249   -2.4260142  -2.4252656   0.829333   -0.7563443  -1.8502816
  0.77651685 -1.7054341  -1.9316062  -4.616432    1.8501337  -0.3053369
  4.767017    0.56259584  0.7324883  -0.3884097  -3.275475   -0.6850633
  3.6221232   0.39064157 -0.49680316  0.80909824  3.9157553   2.3315184
  0.8299782  -1.3997552  -1.1147614   2.2008183  -1.6906853  -1.250305
 -1.2397022  -1.6311872  -0.13397247 -1.3838164  -1.5270644  -0.71716225
 -1.8466197   0.81175977  5.3170447  -0.43428063  0.6254327  -1.6146216
  1.4070415  -1.0383854  -0.21322724 -0.37158608  0.96250975 -1.1169356
 -1.2658123   0.39088786 -1.9702297   0.0642736  -1.6356854   1.7667271
  0.44650987  1.5538126  -3.3357553  -0.68146384 -1.4255176  -1.2646179
 -0.2770887   0.46076667  4.0399265  -2.9

In [0]:
# plot the vectors in 2-D space

pca_2d = PCA(n_components=2)



